In [1]:
import pandas as pd
import pandas as pd
import random
import boto3
import random
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
import joblib
import gzip
import json
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
import pandas as pd
from tqdm import tqdm
import pickle
from keras.preprocessing.text import Tokenizer
import collections
from collections import Counter
import re as regex
from scipy.sparse import csr_matrix
import os
import warnings
warnings.filterwarnings('ignore')
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS
# from gensim import utils
from bs4 import BeautifulSoup as BSHTML
from gensim.parsing.preprocessing import preprocess_string, STOPWORDS #, remove_stopword_tokens
import boto3
import logging
# from ppring import ppring
from botocore.exceptions import ClientError
import requests
from PIL import Image
import sys # to access the system
import cv2
from IPython.display import Image 

C:\Users\fredg\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\fredg\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\fredg\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [ ]:
# input files: 
"datasets/R1_data_indexed.json" 
"json_files/appliances.json.gz"
"json_files/art_crafts.json.gz"
"json_files/beauty.json.gz"
"json_files/cd_vynil.json.gz"
"json_files/garden.gz"
"json_files/grocery.json.gz"
"json_files/luxury.gz"
"json_files/musical.json.gz"
"json_files/pantry.json.gz"
"json_files/pet.json.gz"

# output file:

In [2]:
%%time
# creating random queries of different lengths in different columns
def create_test_data(df, min_query,max_query, label):
    df["title"] = df['title'].apply(lambda x: x.split())
    for len_query in range (min_query,max_query):
        col = "query_{}".format(len_query)
        df[col] = df["title"].apply(lambda x: " ".join(random.sample(x, k=len_query)) if len(x) > len_query else " ".join(x))
    del df["title"]
    df["label"] = label
    return df

# correctly assigning asin indices
def key_pair(asin, asin_index):
    try:
        return int(asin_index[asin])
    except:
        return -1.1

def query_generator():
    
    seed = 30
    n_sample = 1000
    min_query = 3
    max_query = 8
    label = 1
    
    # returns 500 random queries from electronics title dataset
    R1_df = pd.read_json("datasets/R1_data_indexed.json" )
    df_base = R1_df[['asin', 'title','cat_idx']]
    df_complet =df_base.sample(500, random_state = seed)
    query_test = create_test_data(df_complet,min_query,max_query, label)
    print(len(query_test))
    # returns 500 random queries from 10 other Amazon title datasets
    random.seed = seed
    
    # number of samples to download can be hardcoded as n = 50
    liste_files = !ls 'json_files/'
    # n = int(n_sample/2*len(liste_files))
    n = 50
    full_liste = []
    for file in liste_files:
        jsonfilename = 'json_files/{}'.format(file)
        with gzip.open(jsonfilename, 'r') as fin:
             liste_titles = [json.loads(line) for line in fin.readlines()]

        liste_titles = random.sample(liste_titles, n) 


        string = [ x["title"] for x in liste_titles]

        full_liste = full_liste + string
        print(jsonfilename, len(string))

    print(len(full_liste))
    asin = list(range(0,len(full_liste)))
    df_null = pd.DataFrame(zip(asin, full_liste), columns=["asin","title"])
    df_null["cat_idx"] = None
    
    # creates label 0 (no match)
    label = 0
    query_test2 = create_test_data(df_null,min_query,max_query, label)
    
    final_query = pd.concat([query_test,query_test2])
    
    # correctly index asin
    with open('index/R1_asin_index.pickle', 'rb') as filehandle:
            asin_index = pickle.load(filehandle)
    
    
    final_query["asin"] =  final_query["asin"].apply(lambda x: key_pair(x, asin_index)) 
    
    # upload file
    final_query.to_csv("datasets/query_test_1000.csv", index=False)

def initialize():
    #############################
    # Keep True as default
    download = True
    #############################

    if download:
        query_test = pd.read_csv("datasets/query_test_1000.csv")
    else:
        query_generator()
    
    return query_test
        
query_test = initialize()
query_test.head()

CPU times: total: 0 ns
Wall time: 10.1 ms


,asin,cat_idx,query_3,query_4,query_5,query_6,query_7,label
0,69186.0,79.0,"FSK-12JL-W7-1 12W7, Wide",12W7 FSK-12JL-W7-1 Edge (Single),Repair Speaker (Single) JL Edge,"JL Repair Edge Kit, Extra 12W7,","Roll, Repair Extra Wide JL 12W7 12,",1
1,455186.0,620.0,4g Portable Sit,1900 and Mah 3gs,3g 4g Mobile for Cute,4g Kitty 3g and Touch Portable,Mah 3gs Mobile Hello Iphone Kitty Touch,1
2,205033.0,285.0,Sony Bundle and,Sony Bag Camera 32GB,with Gadget Sony (Black) and,SD Bundle 32GB Alpha and Sony,SD a6000 Bundle Alpha (Black) Mirrorless Gadget,1
3,29298.0,21.0,Ft HDMI White,HDMI to White HDMI,4K HDMI 10FT 10 Speed,to Resolution Ft 4K HDMI HDMI,"Cable, to HDMI Video Return Audio HDMI",1
4,360432.0,542.0,Protector E7440 It3,E7440 Screen It3 Guard,Anti E7440 14 Latitude It3,Latitude Guard for It3 Screen Screen,Anti Screen for Screen It3 14 Glare,1


In [3]:
query_test.tail()

,asin,cat_idx,query_3,query_4,query_5,query_6,query_7,label
995,-1.1,NaN,Plush Dog Grriggles,Plush Grriggles Unstuffies Dog,Grriggles Plush Unstuffies Dog Toy,Grriggles Plush Unstuffies Dog Toy,Grriggles Plush Unstuffies Dog Toy,0
996,-1.1,NaN,Pets MidWest 39&quot;,Tall Gate/Pet 39&quot; Pet,Gate/Pet Steel Homes Soft Graphite,for Soft Gate/Pet Gate; 39&quot; Textured,29&quot; &amp; Textured White MidWest in Pet,0
997,-1.1,NaN,Heavy HUG 78,Navy/Light 300g 1200D Heavy,Heavy HUG Navy/Light Horse 1200D,300g Horse Heavy HUG Navy/Light Blue,Heavy Blue Horse Blanket 1200D 78 HUG,0
998,-1.1,NaN,"Bag) Range, Bully",for No - Good,"Range, &hellip; Hand Good -","Range, 100% Dog Inspected Hand -",- USDA/FDA-Approved Natural or Bully Chemicals...,0
999,-1.1,NaN,Trial Dog Labs,Large Trial Labs -,- Trial Large Pack Wapiti,Large Mobility Wapiti Dog Pack Trial,Large Trial Pack Labs Mobility Wapiti Dog,0
